In [2]:
import requests
import json

In [19]:
#Get EcoBee application PIN and tokens

APP_KEY = 'YOUR_API_KEY'
SCOPE = 'smartWrite'

auth_url = 'https://api.ecobee.com/authorize?'
payload = {'response_type':'ecobeePin','client_id': APP_KEY,'scope' : SCOPE}

r = requests.get(auth_url,payload)

r_dict = json.loads(r.text)

print('Your PIN is: ',r_dict['ecobeePin'])

Your PIN is:  z59b


In [21]:
#Get original tokens after PIN validation

#POST https://api.ecobee.com/token?
#        grant_type=ecobeePin&
#        code=AUTHORIZATION_TOKEN&
#        client_id=APP_KEY

auth_code = r_dict['code']
token_url = 'https://api.ecobee.com/token?'
payload = {'grant_type':'ecobeePin','code': auth_code,'client_id': APP_KEY}

r_tokens = requests.post(token_url,params=payload)

r_tokens_dict = json.loads(r_tokens.text)

print(r_tokens_dict)

access_token = r_tokens_dict['access_token']
token_type = r_tokens_dict['token_type']
refresh_token = r_tokens_dict['refresh_token']

print(access_token, token_type, refresh_token)

#Write tokens to file

f=open("tokens.txt","w+")
f.write(access_token)
f.write(",")
f.write(token_type)
f.write(",")
f.write(refresh_token)
f.close

{'expires_in': 3599, 'access_token': '2xRpjOhogkfCoGdwNUncVW5gH1y96OCo', 'refresh_token': '8XmI3m4csdOguojQZubAT0rLShRUKnOo', 'scope': 'smartWrite', 'token_type': 'Bearer'}
2xRpjOhogkfCoGdwNUncVW5gH1y96OCo Bearer 8XmI3m4csdOguojQZubAT0rLShRUKnOo


<function TextIOWrapper.close>

In [23]:
f=open("tokens.txt","r")

token_str = str(f.read())

f.close

token_list = token_str.split(",")

access_token = token_list[0]
token_type = token_list[1]
refresh_token = token_list[2]

print(access_token, refresh_token)

2xRpjOhogkfCoGdwNUncVW5gH1y96OCo 8XmI3m4csdOguojQZubAT0rLShRUKnOo


In [24]:
#Main Program

# set up libraries
# Start loop including time delay
# Set up timekeeping

# 1)  Read tokens from file

f=open("tokens.txt","r")

token_str = str(f.read())

f.close

token_list = token_str.split(",")

access_token = token_list[0]
token_type = token_list[1]
refresh_token = token_list[2]

# 2) Submit old refresh token for valid access token and new refresh token

token_url='https://api.ecobee.com/token?'
payload = {'grant_type':'refresh_token','refresh_token': refresh_token,'client_id': APP_KEY}
    
r_refresh = requests.post(token_url,params=payload)
    
r_refresh_dict = json.loads(r_refresh.text)
    
if r_refresh.status_code == requests.codes.ok:

    token_type = r_refresh_dict['token_type']
    access_token = r_refresh_dict['access_token']
    refresh_token = r_refresh_dict['refresh_token']

    auth = (token_type)+' '+(access_token)

    print(auth)
    
print(r_refresh.status_code)

# 3) Write new tokens back to file

f=open("tokens.txt","w+")
f.write(access_token)
f.write(",")
f.write(refresh_token)
f.close

# 4)  Call EcoBee API to get and parse temperature data into variables

# 5)  Decision logic and set booleans for switches

# 6)  Call smart switch API(s) to set switch states

# 7)  Write variables, states and responses to log file

# 8) Very optional ... plot variables

# 9)  Close loop




Bearer 89Bb6LXYNypDSCbiuqvfdZ4e6Ksw6tcQ
200


<function TextIOWrapper.close>

In [28]:

def get_thermostats(self):

    thermostat_url = 'https://api.ecobee.com/1/thermostat?'

    payload = {'json': ('{"selection":{"selectionType":"registered","includeSensors":"true"}}')}


    header = {'Content-Type': 'application/json;charset=UTF-8','Authorization': auth}


    request = requests.get(thermostat_url,headers=header,params=payload)


    if request.status_code == requests.codes.ok:
            self.authenticated = True
            self.thermostats = request.json()['thermostatList']
            return self.thermostats
        else:
            self.authenticated = False
            logger.info("Error connecting to Ecobee while attempting to get "
                  "thermostat data.  Refreshing tokens and trying again.")
    if self.refresh_tokens():
        return self.get_thermostats()
            else:
                return None

def get_thermostat(self, index):
        ''' Return a single thermostat based on index '''
        return self.thermostats[index]

def get_remote_sensors(self, index):
        ''' Return remote sensors based on index '''
return self.thermostats[index]['remoteSensors']

#request_dict = json.loads(request.text)
#print(request_dict)
#print(request_dict.keys())

IndentationError: unindent does not match any outer indentation level (<ipython-input-28-50ae0ea087bd>, line 19)

In [30]:
self.thermostats = r_thermostat.json()['thermostatList']
print(self.thermostats)

NameError: name 'r_thermostat' is not defined

In [199]:
#not used -- attempt to directly call ecobee API 

import json
import requests

headers = {'Content-Type':'text/json; charset=utf-8',
           'Authorization': access_token}

#POST https://api.ecobee.com/token?
#    grant_type=refresh_token&
#    refresh_token=REFRESH_TOKEN&
#   client_id=APP_KEY

token_url = 'https://api.ecobee.com/token?'
refresh_token = requests.post(token_url,{'grant_type':'refresh_token','refresh_token':'REFRESH TOKEN','client_id': 'tbDjHwLb4cDaKYs4OAat3oImu37e9niu'}

#url = 'https://api.ecobee.com/1/thermostat?'

#payload = {'selectionType':'registered','selectionMatch':'','includeRuntime':'true'}

#r = requests.post(url, headers=headers, json=payload)

#requests.post('https://api.ecobee.com/token?',{'grant_type' : 'refresh_token'},)



if token_r.status_code == 200:
    print('good stuff')
else:

    response_code = token_r.status_code
    print ('error',response_code)
#data=r.get('thermostatList').get('name')
#print(data)

SyntaxError: invalid syntax (<ipython-input-199-293917168b7c>, line 17)

In [193]:
#used to authenticate with EcoBee only

import pyecobee
from pyecobee import *

#dir(EcobeeService)
#help(EcobeeService)
API_key = 'tbDjHwLb4cDaKYs4OAat3oImu37e9niu'
ecobee_service = EcobeeService(thermostat_name='Home',
                               application_key=API_key)

authorize_response = ecobee_service.authorize()
pin = authorize_response.ecobee_pin
code = authorize_response.code
scope = authorize_response.scope
expires_in = authorize_response.expires_in
interval = authorize_response.interval

print(pin, code, scope, expires_in, interval)

#authorize_response = ecobee_service.authorize()
#logger.info(authorize_response.pretty_format())
#logger.info('Authorization Token => {API_key}'.format(ecobee_service.authorization_token))

wngx XyF1TkSHneou6IJcRjr7KgsDOWUxOOwC smartWrite 9 30


In [196]:
import pyecobee

token_response = ecobee_service.refresh_tokens()

thermostat_summary_response = ecobee_service.request_thermostats_summary(selection=Selection(
        selection_type=SelectionType.REGISTERED.value,
        selection_match='',
        include_equipment_status=True))

print(thermostat_summary_response)


[16/Sep/2017 09:59:04] ERROR - EcobeeAuthorizationException raised:
Traceback (most recent call last):
  File "/Users/lenoxpod/anaconda/lib/python3.5/site-packages/pyecobee/service.py", line 155, in __process_http_response
    error_response.error_uri)
pyecobee.exceptions.EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2


EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2

In [192]:
token_response = ecobee_service.refresh_tokens()
print(token_response)

[16/Sep/2017 09:53:51] ERROR - EcobeeAuthorizationException raised:
Traceback (most recent call last):
  File "/Users/lenoxpod/anaconda/lib/python3.5/site-packages/pyecobee/service.py", line 155, in __process_http_response
    error_response.error_uri)
pyecobee.exceptions.EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2


EcobeeAuthorizationException: ecobee authorization error encountered for URL => https://api.ecobee.com/token?grant_type=refresh_token&client_id=tbDjHwLb4cDaKYs4OAat3oImu37e9niu
HTTP error code => 400
Error type => invalid_request
Error description => The request is malformed. Check parameters.
Error URI => https://tools.ietf.org/html/rfc6749#section-5.2

In [182]:
from time import sleep
import datetime

#initialize plug states before loop

office_plug_on = False 
mbed_plug_on = False 
orm_plug_on = False 
lrm_plug_on = False 

while True:
        
# Full payload here for reference, not used  
# selection = Selection(selection_type=SelectionType.REGISTERED.value, selection_match='', include_alerts=True,
#                     include_device=True, include_electricity=False, include_equipment_status=False,
#                    include_events=False, include_extended_runtime=False, include_house_details=False,
#                      include_location=False, include_management=False, include_notification_settings=False,
#                      include_oem_cfg=False, include_privacy=False, include_program=False, include_reminders=False,
#                      include_runtime=False, include_security_settings=False, include_sensors=True,
#                      include_settings=False, include_technician=False, include_utility=False, include_version=False,
#                      include_weather=False)

    #selection of parameters
    
    selection = Selection(selection_type=SelectionType.REGISTERED.value, selection_match='',include_sensors=True)

    #Returns the thermostat_response object including the temperature readings

    thermostat_response = ecobee_service.request_thermostats(selection)

    #convert this response to a string, and then a tuple

    tstat_string = str(thermostat_response)

    tstat_list = tstat_string.split(",")

    tstat_tuple = tuple(tstat_list,)

    #uncomment and use this to determine location of new values
    # e.g. remote sensor readings
    
    x = 0
    
    while x <= 100:
   
        print(x, tstat_tuple[x]) 
    
        x = x+1
    
    # get the values from the tuple

    tstat_name = tstat_tuple[47][7:-1]

    tstat_temp_string = tstat_tuple[53][8:-2]

    #Convert temp string to an number

    tstat_temp = float(tstat_temp_string)/10
    
    #timestamp and time for logic

    timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())

    dectime = float(timestamp[11:13])+float(timestamp[14:16])/60
    
    # Setpoints and logic for smart plug state
    # Temp setpoints in F
    # Time setpoints in decimal hours, 24 hour time
    
    office_temp_upp = 68.0
    office_temp_low = 65.0
    office_on_from_time = 19.0
    office_on_to_time = 8.0
    
    #debug conditionals
    
    #print('temp under lower bound:',tstat_temp <= temp_low)
    #print('time in bounds:',dectime >= on_from_time and dectime <= on_to_time)
    
    if tstat_temp <= office_temp_low and dectime >= office_on_from_time and dectime <= office_on_to_time:
        office_plug_on = True
        office_plug_state = 1
    elif tstat_temp >= office_temp_upp or dectime < office_on_from_time or dectime > office_on_to_time:
        office_plug_on = False
        office_plug_state = 2
    else:
        office_plug_on = office_plug_on
        office_plugstate = 3
    
    #End logic
    
    print(timestamp, tstat_name, tstat_temp, 'Office plug is On? ',office_plug_on)
    
    sleep(300) 

    
    token_response = ecobee_service.refresh_tokens() #refresh token

    #Next step ... Do similar to above for sensors
    #Exception handling to exit the loop & error message
    #logging
    #Link to smart plug
    #plotly?


AttributeError: 'ConnectTimeout' object has no attribute 'message'

In [ ]:
    # logic simulator -- test before using above
    i = 0
    j = 0
    tstat_temp = 65.0
    dectime = 0
    plug_on = True
    temp_upp = 68.0
    temp_low = 65.0
    on_from_time = 1
    on_to_time = 2
    
    for i in range (0,4):
        for j in range (0,6):
     
            if tstat_temp <= temp_low and dectime >= on_from_time and dectime <= on_to_time:
                plug_on = True
            elif tstat_temp >= temp_upp or dectime < on_from_time or dectime > on_to_time:
                plug_on = False
            else:
                plug_on = plug_on
        
            print(dectime, tstat_temp, plug_on)
            tstat_temp = tstat_temp + 1  
            j = j+1
        dectime = dectime + 1
        tstat_temp = 65
        i = i+1
    print('_________________________________')    
    tstat_temp = 70
    dectime = 0
    for i in range (0,4):
        for j in range (0,6):
     
            if tstat_temp <= temp_low and dectime >= on_from_time and dectime <= on_to_time:
                plug_on = True
            elif tstat_temp >= temp_upp or dectime < on_from_time or dectime > on_to_time:
                plug_on = False
            else:
                plug_on = plug_on
        
            print(dectime, tstat_temp, plug_on)
            tstat_temp = tstat_temp - 1  
            j = j+1
        dectime = dectime + 1
        tstat_temp = 70
        i = i+1

In [160]:
timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())

hour = timestamp[12:13]
minute = timestamp[14:16]
decminute = float(minute)/60

print(hour, minute, decminute)

dectime = float(timestamp[12:13])+float(timestamp[14:16])/60 #something is screwy with this math

print(timestamp, dectime)

9 39 0.65
2017-09-10 09:39:20 9.65


In [25]:
#test file write / read for tokens

#Write tokens to file

f=open("tokens.txt","w+")
f.write(access_token)
f.write(",")
f.write(refresh_token)
f.close

access_token="none"
refresh_token="none"

f=open("tokens.txt","r")

token_str = str(f.read())

f.close

token_list = token_str.split(",")

access_token = token_list[0]
refresh_token = token_list[1]

print("access token:", access_token)
print("refresh token:", refresh_token)

access token: 89Bb6LXYNypDSCbiuqvfdZ4e6Ksw6tcQ
refresh token: bTQWwBl4MQvtXw4hhoOlk5rGc1oNP1LF
